In [1]:
# from google.colab import userdata
# tok = userdata.get('git_token')

In [2]:
# !pip install torchmetrics -Uqq
from torchmetrics.classification import BinaryAccuracy

/home/dai/anaconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [51]:
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [52]:
df=pd.read_csv("train.csv")
y=df["smoking"]
df=df.drop(["id", "smoking"],axis=1)

In [53]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [54]:
import torch.nn as nn

class ClsModel(nn.Module):
    def __init__(self, input_size, hidden_size, out_size, dropout_rate=0.3):
        super().__init__()
        self.model = nn.Sequential(
          nn.Linear(in_features=input_size, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(in_features=hidden_size, out_features=out_size, dtype=torch.float64),
          nn.Sigmoid()
      )

    def forward(self, data):
        return self.model(data)


In [55]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = scaler.fit_transform(df)

In [56]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_sample = self.X[idx]
        y_sample = self.y.iloc[idx]
        return x_sample.astype("float64"), y_sample.astype("float64")

In [57]:
def get_data(X,y):
    X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,shuffle=True,stratify=y)
    train_dataset = CustomDataset(X,y)
    val_dataset = CustomDataset(X_test,y_test)
    train_loader = DataLoader(train_dataset,batch_size=128,num_workers=os.cpu_count(),pin_memory=True)
    val_loader = DataLoader(val_dataset,batch_size=32,num_workers=os.cpu_count(),pin_memory=True)
    return train_loader, val_loader

In [58]:
train_loader, val_loader = get_data(df,y)

In [59]:
def save_checkpoint(model, optimizer, epoch, loss, filename, scheduler=None):
    path = filename + '.pth'
    torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'loss': loss,
  }, path)

def load_checkpoint(model, optimizer, scheduler, filename):
    path = filename + '.pth'
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return model, optimizer, scheduler, epoch, loss

In [60]:
torch.manual_seed(42)
model = ClsModel(22,10,1)
model.to(device)
# model.compile()
loss_fn = nn.BCELoss()
adam = torch.optim.Adam(model.parameters(),lr=0.0001)
scheduler = ReduceLROnPlateau(adam, 'min')

In [61]:
# model , adam, epoch, loss = load_checkpoint(model, adam, 'model')
# adam = torch.optim.Adam(model.parameters(),lr=0.01)

In [65]:
for i in range(40):
    #Training
    model.train()
    accumulated_loss = 0
    print(f"Epoch: {i}")
    for idx,batch in enumerate(train_loader):
        msg = f"batch: {idx}"
        print('\r' + msg, end='', flush=True)
        X,y = batch
        X = X.to(device)
        y = y.to(device)
        preds = model(X)
        loss = loss_fn(preds,y.reshape(-1,1))
        adam.zero_grad()
        loss.backward()
        adam.step()
        accumulated_loss += loss.item()
    print()
    _loss = accumulated_loss/len(train_loader)
    scheduler.step(_loss)
    print(accumulated_loss/len(train_loader))
    save_checkpoint(model, adam, i, accumulated_loss/len(train_loader), 'model')

Epoch: 0
batch: 1244
0.487310545812079
Epoch: 1
batch: 1244
0.4869515251801622
Epoch: 2
batch: 1244
0.4861162787874692
Epoch: 3
batch: 1244
0.4851529544124094
Epoch: 4
batch: 1244
0.4874720139515642
Epoch: 5
batch: 1244
0.4872738078277307
Epoch: 6
batch: 1244
0.48745212482928424
Epoch: 7
batch: 1244
0.4862407673135677
Epoch: 8
batch: 1244
0.4881104303580053
Epoch: 9
batch: 1244
0.4878195446483235
Epoch: 10
batch: 1244
0.4870770000038138
Epoch: 11
batch: 1244
0.4874381357753385
Epoch: 12
batch: 1244
0.48679982959391155
Epoch: 13
batch: 1244
0.48682101244434944
Epoch: 14
batch: 1244
0.4865833429295426
Epoch: 15
batch: 1244
0.48664480883986627
Epoch: 16
batch: 1244
0.4862553482495386
Epoch: 17
batch: 1244
0.48654061824293526
Epoch: 18
batch: 1244
0.4846041616323142
Epoch: 19
batch: 1244
0.48579555174444944
Epoch: 20
batch: 1244
0.4858317983554645
Epoch: 21
batch: 1244
0.48745469964305366
Epoch: 22
batch: 1244
0.48554665956029014
Epoch: 23
batch: 1244
0.4861227016858274
Epoch: 24
batch: 12

In [ ]:
# os.cpu_count()

In [ ]:
# df=pd.read_csv("/content/train.csv")
# y=df["smoking"]
# df=df.drop(["id", "smoking"],axis=1)
# X_train,X_test,y_train,y_test = train_test_split(df,y,random_state=42,shuffle=True,stratify=y)

In [ ]:
# test_X = torch.Tensor(X_test.to_numpy()).to(torch.float64)
# test_y = torch.Tensor(y_test.to_numpy()).to(torch.float64)

In [ ]:
# metrics = BinaryAccuracy().to(device)

In [ ]:
# with torch.inference_mode():
#     preds = model(test_X.to(device))
#     acc = metrics(preds,test_y.reshape(-1,1).to(device))
#     print(acc)

In [ ]:
save_checkpoint(model,adam,40,0.457, "model3")

In [ ]:
test = pd.read_csv("test.csv",index_col="id")
test = scaler.transform(test)
test = torch.Tensor(test).to(torch.float64)

In [ ]:
# with torch.inference_mode():
#     preds = model(test_X.to(device))
#     acc = metrics(preds,test_y.reshape(-1,1).to(device))
#     print(acc)

In [ ]:
with torch.inference_mode():
    preds = model(test.to(device))
    print(preds)

tensor([[0.5624],
        [0.4480],
        [0.4027],
        ...,
        [0.5584],
        [0.1953],
        [0.1139]], dtype=torch.float64)


In [ ]:
submission = pd.read_csv("sample_submission.csv", index_col="id")
submission['smoking'] = preds.cpu()

In [ ]:
submission.to_csv("submission.csv")